In [54]:
import urllib.request

import numpy as np
import pandas as pd

import cvxpy as cp

In [46]:
url = "http://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/port5.txt"
data = urllib.request.urlopen(url)
num_entries = 0

mean_rr = []
std_rr = []

for i, line in enumerate(data):
    if i == 0:
        num_entries = int(line)
        corr_mat = np.zeros((num_entries, num_entries))
    elif i <= num_entries:
        mean, std = line.split()
        mean_rr.append(float(mean))
        std_rr.append(float(std))
    else:
        i, j, corr = line.split()
        corr_mat[int(i) - 1, int(j) - 1] = float(corr)
        corr_mat[int(j) - 1, int(i) - 1] = float(corr)

In [48]:
mean_rr = np.array(mean_rr)
std_rr = np.array(std_rr)
print(f"mu: {mean_rr.shape}, sigma: {std_rr.shape}, corr: {corr_mat.shape}")

mu: (225,), sigma: (225,), corr: (225, 225)


1. $\dfrac{1}{2}x^TCx$ represents portfolio risk because we're summing over covariances between rates of returns which gets us an overall variance
2. Because $\mu$ is mean rate of return so any $\mu_ix_i \geq r, \mu_i, x_i \geq 0 \implies \mu^Tx\geq r$
3. Since we cannot have negative owning of some stock and all the investments should add up to the capital we have

### Covariance matrix

In [51]:
cov = np.zeros((num_entries, num_entries))

for i in range(num_entries):
    for j in range(num_entries):
        cov[i, j] = corr_mat[i, j] * std_rr[i] * std_rr[j]

### Quadratic problem

In [62]:
x = cp.Variable(num_entries)
r = 0.002

prob = cp.Problem(cp.Minimize(cp.quad_form(x, cov)),
                 [
                     mean_rr @ x >= r,
                     np.sum(x) == 1,
                     x >= 0,
                 ])
prob.solve()

print("\nThe optimal value is", prob.value)
print("A solution x is")
print(x.value)
print("A dual solution corresponding to the inequality constraints is")
print(prob.constraints[0].dual_value)


The optimal value is inf
A solution x is
None
A dual solution corresponding to the inequality constraints is
None
